In [ ]:
# 
# Build pCVR using GAP bigquery and Tensorflow DNN library 
# Used calibration period for training & holdout period for validation
# Script can be run on Compute Engine of GCP
# The query to extract GA360 features referred to an Auto case by Yiling Liu(yilliu@) 
# 
# By JeeWook Kim
#

<h1> Install libraries

In [1]:
%%bash
pip install --upgrade pip
pip install --upgrade google-api-python-client
pip install --upgrade gcloud
pip install tensorflow
pip install --ignore-installed --upgrade tensorflow==1.9.0


Requirement already up-to-date: pip in /usr/local/lib/python2.7/dist-packages (18.0)
Requirement already up-to-date: google-api-python-client in /usr/local/lib/python2.7/dist-packages (1.7.4)
Requirement already up-to-date: gcloud in /usr/local/lib/python2.7/dist-packages (0.18.3)
  Using cached https://files.pythonhosted.org/packages/37/ff/97d4542f805ae25bf4b65b6263515584c78bd9a6111ed78ea971eff2946a/tensorflow-1.9.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/e6/35/f187bdf23be87092bd0f1200d43d23076cee4d0dec109f195173fd3ebc79/mock-2.0.0-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fa/53/685b9d68e5734cf8e2d13b2d4d29e6cede94714b665019a5627a623b3e21/grpcio-1.13.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/85/51/ba4564ded90e093dbb6adfc3e21f99ae953d9ad56477e1b0d4a93bacf7d3/numpy-1.15.0-cp27-cp27mu-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packag

google-cloud 0.27.0 has requirement google-cloud-core<0.27dev,>=0.26.0, but you'll have google-cloud-core 0.27.1 which is incompatible.
google-cloud 0.27.0 has requirement google-cloud-storage<1.4dev,>=1.3.0, but you'll have google-cloud-storage 1.4.0 which is incompatible.
google-cloud-vision 0.26.0 has requirement google-cloud-core<0.27dev,>=0.26.0, but you'll have google-cloud-core 0.27.1 which is incompatible.
google-cloud-vision 0.26.0 has requirement google-gax<0.16dev,>=0.15.13, but you'll have google-gax 0.12.5 which is incompatible.
google-cloud-logging 1.2.0 has requirement google-cloud-core<0.27dev,>=0.26.0, but you'll have google-cloud-core 0.27.1 which is incompatible.
google-cloud-speech 0.28.0 has requirement google-cloud-core<0.27dev,>=0.26.0, but you'll have google-cloud-core 0.27.1 which is incompatible.
google-cloud-speech 0.28.0 has requirement google-gax<0.16dev,>=0.15.13, but you'll have google-gax 0.12.5 which is incompatible.
google-cloud-videointelligence 0.25.

In [2]:
import tensorflow as tf
print(tf.__version__)

1.9.0


/usr/local/lib/python2.7/dist-packages/scipy/ndimage/measurements.py:36: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from . import _ni_label
/usr/local/lib/python2.7/dist-packages/scipy/ndimage/measurements.py:36: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  from . import _ni_label


<h1> Prepare dataset using GA360

In [31]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib

import numpy as np
import tensorflow as tf
print(tf.__version__)

import datetime
from dateutil.relativedelta import relativedelta


# Dataset should look like this 
training_set_data = np.array([[6.4, 2.8, 5.6, 2.2],
                            [5.,  2.3, 3.3, 1. ],
                            [4.9, 2.5, 4.5, 1.7],
                            [4.9, 3.1, 1.5, 0.1],
                            [5.7, 3.8, 1.7, 0.3],
                            [4.4, 3.2, 1.3, 0.2],
                            [5.4, 3.4, 1.5, 0.4],
                            [6.9, 3.1, 5.1, 2.3],
                            [6.7, 3.1, 4.4, 1.4],
                            [5.1, 3.7, 1.5, 0.4]])
training_set_target = np.array([2, 1, 2, 0, 0, 0, 0, 2, 1, 0])

print('# training_set_data')
print(np.array(training_set_data)[:10])
print('# training_set_target')
print(np.array(training_set_target)[:10])

test_set_data = np.array([[5.9, 3.,  4.2, 1.5],
                        [6.9, 3.1, 5.4, 2.1],
                        [5.1, 3.3, 1.7, 0.5],
                        [6.,  3.4, 4.5, 1.6],
                        [5.5, 2.5, 4.,  1.3],
                        [6.2, 2.9, 4.3, 1.3],
                        [5.5, 4.2, 1.4, 0.2],
                        [6.3, 2.8, 5.1, 1.5],
                        [5.6, 3.,  4.1, 1.3],
                        [6.7, 2.5, 5.8, 1.8]])
test_set_target = np.array([1, 2, 0, 1, 1, 1, 0, 2, 1, 2])  
print('# test_set_data')
print(np.array(test_set_data)[:10])
print('# test_set_target')
print(np.array(test_set_target)[:10])

today = datetime.date.today().strftime("%Y%m%d")
# calibration begin date => 12 months ago
begin_date = '20160801'
# obserbation end date => 3 days ago
end_date = '20170801'
# calibration end date => 6 months ago
# calibration_end_date = (datetime.date.today() + relativedelta(months=-6)).strftime("%Y%m%d")
calibration_end_date = '20170201'
# Animals In Space table
# gap_table = 'google.com:bigquery-150208.90624960.ga_sessions_*'
# Googel Store demo table
gap_table = 'bigquery-public-data.google_analytics_sample.ga_sessions_*'


print('# today: {}'.format(today))
print('# begin_date: {}'.format(begin_date))
print('# end_date: {}'.format(end_date))
print('# calibration_end_date: {}'.format(calibration_end_date))
# query to retrieve GAP exported BigQuery ecommerce tranactions (users with purchases in the calibration period)
sql_train = """ 
          WITH ga_raw AS ( -- hit level data
          SELECT
            date,
            fullVisitorId,
            channelGrouping,
            socialEngagementType,
            visitId,
            visitNumber,
            trafficSource.source,
            trafficSource.medium,
            device.deviceCategory,
            device.browser,
            hits.hitNumber,
            hits.page.pagePath as pagePath,
            hits.type as type,
            hits.eventInfo.eventCategory as eventCategory,
            hits.eventInfo.eventAction as eventAction,
            hits.eventInfo.eventLabel as eventLabel,
            TIMESTAMP_SECONDS(visitStartTime) AS sessionStartTtime, 
            TIMESTAMP_ADD(TIMESTAMP_SECONDS(visitStartTime), INTERVAL hits.time MILLISECOND) AS hitTime,
            CASE WHEN hits.hour IN (5,6,7,8,9,10) THEN 1 ELSE 0 END AS morningVisit,
            CASE WHEN hits.hour IN (11,12,13,14,15,16) THEN 1 ELSE 0 END AS dayVisit,
            CASE WHEN hits.hour IN (17,18,19,20,21,22) THEN 1 ELSE 0 END AS eveningVisit,
            totals.timeOnSite AS timeOnSite,
            totals.bounces AS bounceNumber,
            totals.timeOnScreen	AS timeOnScreen,
            totals.transactions AS transactions,
            IF(REGEXP_CONTAINS(trafficSource.source , 'google') AND REGEXP_CONTAINS(trafficSource.medium , 'cpc'), 1, 0) AS fromPaidSearch,
            IF(REGEXP_CONTAINS(trafficSource.source , 'google') AND REGEXP_CONTAINS(trafficSource.medium , 'organic'), 1, 0) AS fromOrganicSearch
          FROM `"""+gap_table+"""`, unnest(hits) as hits
          WHERE
           (_TABLE_SUFFIX >= '"""+begin_date+"""' AND _TABLE_SUFFIX <= '"""+calibration_end_date+"""') ),
       
        session AS ( -- aggregate hit level to session level
          SELECT fullVisitorId, visitId,
            SUM(if (timeOnSite is null, 0, timeOnSite)) AS timeOnSite,
            SUM(if (bounceNumber is null, 0, bounceNumber)) AS bounceNumber,
            SUM( dayVisit ) AS dayVisit,
            SUM( eveningVisit ) AS eveningVisit ,
            SUM( morningVisit ) AS morningVisit ,
            ANY_VALUE(visitNumber) as visitNumber,

            SUM(if( type = 'PAGE', 1, 0)) as totalPageViews, 
            SUM(if( type = 'EVENT', 1, 0)) as totalEvents, 

            SUM(if(pagePath LIKE '%/apparel%', 1, 0)) as apparelViews,
            SUM(if(pagePath LIKE '%/bags%', 1, 0)) as bagsViews,
            SUM(if(pagePath LIKE '%/drinkware%', 1, 0)) as drinkwareViews,
            SUM(if(pagePath LIKE '%/accessories%', 1, 0)) as accessoriesViews,
            SUM(if(pagePath LIKE '%/office%', 1, 0)) as officeViews,

            IF(SUM(fromPaidSearch) != 0, 1, 0) AS  fromPaidSearch,
            IF(SUM(fromOrganicSearch) != 0, 1, 0) AS  fromOrganicSearch,

            IF(SUM(transactions) is null, False, True) AS hasConverted,

            COUNT(*) as totalInteractions    
          FROM ga_raw
          GROUP BY fullVisitorId , visitId ),
  
       ml_dataset AS ( -- aggregate seesion level data to user level

          SELECT 
            fullVisitorId, 
            MAX(visitNumber) as totalSessions,
            SUM(totalPageViews) as totalPageViews, 
            SUM(totalInteractions) as totalInteractions, 

            SUM(timeOnSite) AS timeOnSite,
            SUM(bounceNumber) AS bounceNumber,
            SUM( dayVisit ) AS dayVisit,
            SUM( eveningVisit ) AS eveningVisit ,
            SUM( morningVisit ) AS morningVisit ,

            SUM(totalEvents) as totalEvents, 

            SUM(apparelViews) as apparelViews,
            SUM(bagsViews) as bagsViews,
            SUM(drinkwareViews)  as drinkwareViews,
            SUM(accessoriesViews) as accessoriesViews,
            SUM(officeViews) as officeViews,

            SUM(fromPaidSearch) AS fromPaidSearch,
            SUM(fromOrganicSearch) AS fromOrganicSearch,
            ANY_VALUE(hasConverted) AS hasConverted

          FROM session
          GROUP BY fullVisitorId)

        select * from ml_dataset; 
"""

print ('# BigQuery SQL - train data')  
print (sql_train)
    
sql_test = """ 
         WITH ga_raw AS (
          SELECT
            date,
            fullVisitorId,
            channelGrouping,
            socialEngagementType,
            visitId,
            visitNumber,
            trafficSource.source,
            trafficSource.medium,
            device.deviceCategory,
            device.browser,
            hits.hitNumber,
            hits.page.pagePath as pagePath,
            hits.type as type,
            hits.eventInfo.eventCategory as eventCategory,
            hits.eventInfo.eventAction as eventAction,
            hits.eventInfo.eventLabel as eventLabel,
            TIMESTAMP_SECONDS(visitStartTime) AS sessionStartTtime, 
            TIMESTAMP_ADD(TIMESTAMP_SECONDS(visitStartTime), INTERVAL hits.time MILLISECOND) AS hitTime,
            CASE WHEN hits.hour IN (5,6,7,8,9,10) THEN 1 ELSE 0 END AS morningVisit,
            CASE WHEN hits.hour IN (11,12,13,14,15,16) THEN 1 ELSE 0 END AS dayVisit,
            CASE WHEN hits.hour IN (17,18,19,20,21,22) THEN 1 ELSE 0 END AS eveningVisit,
            totals.timeOnSite AS timeOnSite,
            totals.bounces AS bounceNumber,
            totals.timeOnScreen	AS timeOnScreen,
            totals.transactions AS transactions,
            IF(REGEXP_CONTAINS(trafficSource.source , 'google') AND REGEXP_CONTAINS(trafficSource.medium , 'cpc'), 1, 0) AS fromPaidSearch,
            IF(REGEXP_CONTAINS(trafficSource.source , 'google') AND REGEXP_CONTAINS(trafficSource.medium , 'organic'), 1, 0) AS fromOrganicSearch
          FROM `"""+gap_table+"""`, unnest(hits) as hits
          WHERE
           (_TABLE_SUFFIX > '"""+calibration_end_date+"""' AND _TABLE_SUFFIX <= '"""+end_date+"""') ),
       
        session AS ( -- aggregate hit level to session level
          SELECT fullVisitorId, visitId,
            SUM(if (timeOnSite is null, 0, timeOnSite)) AS timeOnSite,
            SUM(if (bounceNumber is null, 0, bounceNumber)) AS bounceNumber,
            SUM( dayVisit ) AS dayVisit,
            SUM( eveningVisit ) AS eveningVisit ,
            SUM( morningVisit ) AS morningVisit ,
            ANY_VALUE(visitNumber) as visitNumber,

            SUM(if( type = 'PAGE', 1, 0)) as totalPageViews, 
            SUM(if( type = 'EVENT', 1, 0)) as totalEvents, 

            SUM(if(pagePath LIKE '%/apparel%', 1, 0)) as apparelViews,
            SUM(if(pagePath LIKE '%/bags%', 1, 0)) as bagsViews,
            SUM(if(pagePath LIKE '%/drinkware%', 1, 0)) as drinkwareViews,
            SUM(if(pagePath LIKE '%/accessories%', 1, 0)) as accessoriesViews,
            SUM(if(pagePath LIKE '%/office%', 1, 0)) as officeViews,

            IF(SUM(fromPaidSearch) != 0, 1, 0) AS  fromPaidSearch,
            IF(SUM(fromOrganicSearch) != 0, 1, 0) AS  fromOrganicSearch,

            IF(SUM(transactions) is null, False, True) AS hasConverted,

            COUNT(*) as totalInteractions    
          FROM ga_raw
          GROUP BY fullVisitorId , visitId ),
  
       ml_dataset AS ( -- aggregate seesion level data to user level

          SELECT 
            fullVisitorId, 
            MAX(visitNumber) as totalSessions,
            SUM(totalPageViews) as totalPageViews, 
            SUM(totalInteractions) as totalInteractions, 

            SUM(timeOnSite) AS timeOnSite,
            SUM(bounceNumber) AS bounceNumber,
            SUM( dayVisit ) AS dayVisit,
            SUM( eveningVisit ) AS eveningVisit ,
            SUM( morningVisit ) AS morningVisit ,

            SUM(totalEvents) as totalEvents, 

            SUM(apparelViews) as apparelViews,
            SUM(bagsViews) as bagsViews,
            SUM(drinkwareViews)  as drinkwareViews,
            SUM(accessoriesViews) as accessoriesViews,
            SUM(officeViews) as officeViews,

            SUM(fromPaidSearch) AS fromPaidSearch,
            SUM(fromOrganicSearch) AS fromOrganicSearch,
            ANY_VALUE(hasConverted) AS hasConverted

          FROM session
          GROUP BY fullVisitorId)

        select * from ml_dataset; 
"""

print ('# BigQuery SQL - test data')  
print (sql_test)
    
# execute the query using datalab lib
import google.datalab.bigquery as bq
# Pandas lib to handle table data
import pandas as pd
transaction_query = bq.Query(sql_train)
query_result = transaction_query.execute()
query_data = query_result.result().to_dataframe()
    
print('# query_data # of converted users {}'.format(query_data[query_data['hasConverted']==True].shape[0]))
print(query_data[query_data['hasConverted']==True].head(20))
print('# query_data # of not converted users {}'.format(query_data[query_data['hasConverted']==False].shape[0]))
print(query_data[query_data['hasConverted']==False].head(20))
    
training_set_data = query_data.as_matrix(columns=["totalPageViews", "totalInteractions", "timeOnSite","bounceNumber","dayVisit","eveningVisit","morningVisit","totalEvents","apparelViews","bagsViews","drinkwareViews","accessoriesViews","officeViews","fromPaidSearch","fromOrganicSearch"])
training_set_target = query_data.as_matrix(columns=["hasConverted"])
    
    
test_transaction_query = bq.Query(sql_test)
test_query_result = test_transaction_query.execute()
test_query_data = test_query_result.result().to_dataframe()
    


1.9.0
# training_set_data
[[6.4 2.8 5.6 2.2]
 [5.  2.3 3.3 1. ]
 [4.9 2.5 4.5 1.7]
 [4.9 3.1 1.5 0.1]
 [5.7 3.8 1.7 0.3]
 [4.4 3.2 1.3 0.2]
 [5.4 3.4 1.5 0.4]
 [6.9 3.1 5.1 2.3]
 [6.7 3.1 4.4 1.4]
 [5.1 3.7 1.5 0.4]]
# training_set_target
[2 1 2 0 0 0 0 2 1 0]
# test_set_data
[[5.9 3.  4.2 1.5]
 [6.9 3.1 5.4 2.1]
 [5.1 3.3 1.7 0.5]
 [6.  3.4 4.5 1.6]
 [5.5 2.5 4.  1.3]
 [6.2 2.9 4.3 1.3]
 [5.5 4.2 1.4 0.2]
 [6.3 2.8 5.1 1.5]
 [5.6 3.  4.1 1.3]
 [6.7 2.5 5.8 1.8]]
# test_set_target
[1 2 0 1 1 1 0 2 1 2]
# today: 20180802
# begin_date: 20160801
# end_date: 20170801
# calibration_end_date: 20170201
# BigQuery SQL - train data
 
          WITH ga_raw AS (
          SELECT
            date,
            fullVisitorId,
            channelGrouping,
            socialEngagementType,
            visitId,
            visitNumber,
            trafficSource.source,
            trafficSource.medium,
            device.deviceCategory,
            device.browser,
            hits.hitNumber,
          

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:320: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:321: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/pcvr_model/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [32]:
    
test_set_data = test_query_data.as_matrix(columns=["totalPageViews", "totalInteractions", "timeOnSite","bounceNumber","dayVisit","eveningVisit","morningVisit","totalEvents","apparelViews","bagsViews","drinkwareViews","accessoriesViews","officeViews","fromPaidSearch","fromOrganicSearch"])
test_set_target = test_query_data.as_matrix(columns=["hasConverted"])
    
# Specify that all features have real-value data
feature_columns = [tf.feature_column.numeric_column("x", shape=[15])]

# Build 3 layer DNN with 10, 20, 10 units respectively.
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                        hidden_units=[10, 20, 10],
                                        n_classes=2,
                                        model_dir="/tmp/pcvr_model")
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(training_set_data)},
    y=np.array(training_set_target),
    num_epochs=None,
    shuffle=True)

# Train model.
classifier.train(input_fn=train_input_fn, steps=2000)

# Define the test inputs
test_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": np.array(test_set_data)},
    y=np.array(test_set_target),
    num_epochs=1,
    shuffle=False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))

# Classify samples.
new_input = np.array(
    [[6.4, 3.2, 4.5, 1.5],
    [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)
new_input= test_query_data[query_data['hasConverted']==False].as_matrix(columns=["totalPageViews", "totalInteractions", "timeOnSite","bounceNumber","dayVisit","eveningVisit","morningVisit","totalEvents","apparelViews","bagsViews","drinkwareViews","accessoriesViews","officeViews","fromPaidSearch","fromOrganicSearch"])
new_input_key = test_query_data[query_data['hasConverted']==False].as_matrix(columns=["fullVisitorId"])
predict_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": new_input},
    num_epochs=1,
    shuffle=False)

predictions = list(classifier.predict(input_fn=predict_input_fn))
    


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_task_type': 'worker', '_global_id_in_cluster': 0, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fba34eee310>, '_evaluation_master': '', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_num_ps_replicas': 0, '_tf_random_seed': None, '_master': '', '_device_fn': None, '_num_worker_replicas': 1, '_task_id': 0, '_log_step_count_steps': 100, '_model_dir': '/tmp/pcvr_model', '_train_distribute': None, '_save_summary_steps': 100}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/pcvr_model/model.ckpt-4000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tenso

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:39: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/pcvr_model/model.ckpt-6000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [56]:
for key in new_input_key[:5,:]:
  fullVisitorId = key[0]
  print("\nfullVisitorId: {}".format(fullVisitorId))

for p in predictions[:5]:
  print (p)
  pred = p['probabilities']
  print(pred)
  prob = p['probabilities'][1]
  print(prob)
p_array = np.empty((0, 1))
for prob in predictions:
  probability = prob['probabilities'][1]*100
  new_row = np.array([[probability]])
  p_array = np.vstack((p_array, new_row))
  
for key, prob in zip(new_input_key[:5,:], predictions[:5]):
    probability = prob['probabilities'][1] # probabilty of conversion (class id = 1)
    fullVisitorId = key[0]
    print("\nfullVisitorId: {}, probability: {:.1f}".format(fullVisitorId,
                          100 * probability))
#p = np.array(prob['probabilities'][1] for prob in predictions)
print(p_array.shape)
print(new_input_key.shape)
df = pd.DataFrame(np.hstack((new_input_key, p_array)), columns=['fullVisitorId','probability'])
#df = pd.DataFrame(np.hstack((new_input_key, predictions(['probabilities'][1]))), columns=['fullVisitorId','probability'])

print(df.head())


fullVisitorId: 5900472703847352092

fullVisitorId: 0424668908545837304

fullVisitorId: 5484016666945224510

fullVisitorId: 6719182032425610749

fullVisitorId: 6343719132544553875
{'probabilities': array([1.00000000e+00, 1.15154375e-08], dtype=float32), 'logits': array([-18.279577], dtype=float32), 'classes': array(['0'], dtype=object), 'class_ids': array([0]), 'logistic': array([1.1515438e-08], dtype=float32)}
[1.00000000e+00 1.15154375e-08]
1.15154375e-08
{'probabilities': array([1.00000000e+00, 1.15154375e-08], dtype=float32), 'logits': array([-18.279577], dtype=float32), 'classes': array(['0'], dtype=object), 'class_ids': array([0]), 'logistic': array([1.1515438e-08], dtype=float32)}
[1.00000000e+00 1.15154375e-08]
1.15154375e-08
{'probabilities': array([1.00000000e+00, 1.15154375e-08], dtype=float32), 'logits': array([-18.279577], dtype=float32), 'classes': array(['0'], dtype=object), 'class_ids': array([0]), 'logistic': array([1.1515438e-08], dtype=float32)}
[1.00000000e+00 1.151